<a href="https://colab.research.google.com/github/Jeevesh8/arg_mining/blob/main/notebooks/sMLM_LF_pred_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [7]:
from transformers import LongformerTokenizer, LongformerForMaskedLM

In [5]:
tokenizer = LongformerTokenizer.from_pretrained('Jeevesh8/sMLM-LF')

In [8]:
model = LongformerForMaskedLM.from_pretrained('Jeevesh8/sMLM-LF')

Downloading:   0%|          | 0.00/896 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/567M [00:00<?, ?B/s]

In [42]:
import numpy as np

In [63]:
def get_global_attention_mask(tokenized_threads: np.ndarray) -> np.ndarray:
    """Returns an attention mask, with 1 where there are [USER{i}] tokens and 
    0 elsewhere.
    """
    mask = np.zeros_like(tokenized_threads)
    for user_token in ["[UNU]"]+[f"[USER{i}]" for i in range(10)]:
        user_token_id = tokenizer.encode(user_token)[1:-1]
        mask = np.where(tokenized_threads==user_token_id, 1, mask)
    return np.array(mask, dtype=bool)

In [87]:
txt = """[USER0] CMV: I feel skill is largely determined by experience. Compliments on skill are almost meaningless. [NEWLINE] In high school, I thought I was "good at math" as I'm the son of a math teacher and electrical engineer . [NEWLINE] In college, I learned that math was not something you're "good at" but something you have to put hard work into and is almost the sole determiner in the level of skill you obtain . [NEWLINE] So then isn't almost any compliment almost to be expected? I've spent a lot of time with similar problems -- how could I not know all the details and little tricks of these problems? [NEWLINE] I feel a compliment recognizes something given : I feel everyone is passionate about something , whether it be math or psychology or medicine ( I don't hear "you're so good at biology" but I think I should ). [NEWLINE] [USER1] Then wouldn't a complement be just an acknowledgement of the time and effort you put into something that most people see as hard or worthwhile? [NEWLINE] This implies the complement is meaningful.[NEWLINE] ( Most people don't do this - either they don't put the time and effort into something generally hard or worthwhile or the time and effort isn't hard or worthwhile .) [NEWLINE] [USER0] When I receive a compliment I should publicly recognize that then, right? [NEWLINE] No only am I accepting their compliment , I'm showing some of what I truly believe. That counts as a &#8710;! </s> [USER0] said So then isn't almost any compliment almost to be expected? <mask> <mask> <mask> [USER1] said Most people don't do this"""

In [88]:
encoding = tokenizer(txt)

In [89]:
encoding["input_ids"] = torch.tensor([encoding["input_ids"]])

In [90]:
global_attention_mask = get_global_attention_mask(np.array(encoding["input_ids"]))

In [93]:
model_out = model(input_ids=encoding["input_ids"], global_attention_mask=torch.tensor(global_attention_mask))

In [94]:
import torch

In [95]:
max_out = torch.max(model_out.logits[encoding["input_ids"]==tokenizer.mask_token_id], dim=-1)

In [97]:
tokenizer.decode(max_out[1])

' I disagree I'